In [20]:
include("./src/parse_file.jl")

Completed analyis of 63 reads in chr1
Completed analyis of 37 reads in chr2


Dict{String, SparseMatrixCSC{Int64, Int64}} with 2 entries:
  "chr2" => sparse([2, 2, 4, 6, 2, 4, 6, 4, 6, 4  …  15, 15, 16, 16, 17, 18, 17…
  "chr1" => sparse([2, 2, 2, 2, 2, 3, 3, 1, 1, 4  …  23, 24, 25, 26, 27, 21, 22…

In [21]:
path = "./testing/test_epiread_small.bed"
big_bgz_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/test_epiread.bed.bgz"
part_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"


"/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"

In [22]:
results = analyze_file(path)

Completed analyis of 63 reads in chr1
Completed analyis of 37 reads in chr2


Dict{String, SparseMatrixCSC{Int64, Int64}} with 2 entries:
  "chr2" => sparse([2, 2, 4, 6, 2, 4, 6, 4, 6, 4  …  15, 15, 16, 16, 17, 18, 17…
  "chr1" => sparse([2, 2, 2, 2, 2, 3, 3, 1, 1, 4  …  23, 24, 25, 26, 27, 21, 22…

In [38]:
x= results["chr1"]

27×631931 SparseMatrixCSC{Int64, Int64} with 105 stored entries:
⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠈⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠒⠂⠐

In [27]:
@time x[2,:]

  0.000547 seconds (5 allocations: 320 bytes)


631931-element SparseVector{Int64, Int64} with 5 stored entries:
  [136291]  =  77
  [136312]  =  77
  [136326]  =  77
  [136328]  =  77
  [136340]  =  77

In [28]:
function drop_zero_rows(matrix::Matrix{Int64})
	rows_to_keep = Vector{Int64}()
	sizehint!(rows_to_keep, size(matrix)[1])
	for row in 1:size(matrix)[1]
		if sum(matrix[row,:]) != 0
			push!(rows_to_keep, row)
		end
	end
	return matrix[rows_to_keep,:]
end

drop_zero_rows (generic function with 1 method)

In [35]:
@profview drop_zero_rows(Matrix(x[:,130000:138000]))

┌ Warning: There were no samples collected.
│ Run your program longer (perhaps by running it multiple times),
│ or adjust the delay between samples with `Profile.init()`.
└ @ Profile /Users/administrator/.julia/scratchspaces/a66863c6-20e8-4ff4-8a62-49f30b1f605e/agent-cache/default-macmini-aarch64-1.0/build/default-macmini-aarch64-1-0/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Profile/src/Profile.jl:1208


In [37]:
version()

UndefVarError: UndefVarError: version not defined